## Importing libraries

In [1]:
import keras
import os
import pickle
import tensorflow as tf
import pandas as pd

from keras import callbacks
from keras.saving import save_model
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, Reshape, GlobalAveragePooling2D, GlobalMaxPooling2D, Multiply, Concatenate

## Setting up meta parameters

In [2]:
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
BATCH_SIZE = 128
SEED = 99
EPOCHS = 10
LABEL_CLASS = 26
IMAGE_CHANNELS= 3
LEARNINGRATE = 1e-5
WEIGHTS = "vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"

start_epoch = 0
dataset_dir = os.path.abspath("..\\Datasets\\TrainingDatasets")
checkpoints_dir = os.path.abspath("TrainingCheckpoints\\CBAM")
checkpoints_path = os.path.join(checkpoints_dir, "cp-{epoch:04d}.keras")
datagen = ImageDataGenerator(rescale=1.0/255)
data_gen_args = dict(directory=dataset_dir, x_col='images', y_col='labels', target_size=IMAGE_SIZE, class_mode='categorical', batch_size=BATCH_SIZE, seed = SEED)
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience = 5, verbose = 1, restore_best_weights=True)
learningrate_reduction = callbacks.ReduceLROnPlateau(monitor='val_loss', patience= 2, verbose= 1)

## Attention definition

In [3]:
class Channel_Attention(layers.Layer):
    def __init__(self, channels, reduction, cbam_id):
        super().__init__(name=f"cbam{cbam_id}_channel")

        # Property for saving last attention map
        self.channel_attention_map = None

        self.avg_pool = GlobalAveragePooling2D(name=f"cbam{cbam_id}_channel_global_avg_pool")
        self.max_pool = GlobalMaxPooling2D(name=f"cbam{cbam_id}_channel_global_max_pool")

        self.dense1 = Dense(channels // reduction, activation='relu', name=f"cbam{cbam_id}_channel_dense1")
        self.dense2 = Dense(channels, activation=None, name=f"cbam{cbam_id}_channel_dense2")

        self.reshape = Reshape((1, 1, channels), name=f"cbam{cbam_id}_channel_reshape")
        self.multiply = Multiply(name=f"cbam{cbam_id}_channel_multiply")

    def build(self, input_shape):
        x_shape = input_shape
        
        self.avg_pool.build(input_shape)
        self.avg_pool.build(input_shape)
        x_shape = self.avg_pool.compute_output_shape(input_shape)

        self.dense1.build(x_shape)
        x_shape = self.dense1.compute_output_shape(x_shape)

        self.dense2.build(x_shape)
        x_shape = self.dense2.compute_output_shape(x_shape)
    
    def compute_output_shape(self, input_shape):
        return input_shape

    def call(self, input, save_attention=False):
        x = input

        avg_pool = self.avg_pool(x)
        avg_pool = self.dense1(avg_pool)
        avg_pool = self.dense2(avg_pool)
        
        max_pool = self.max_pool(x)
        max_pool = self.dense1(max_pool)
        max_pool = self.dense2(max_pool)

        x = avg_pool + max_pool
        x = tf.nn.sigmoid(x)
        x = self.reshape(x)

        if save_attention:
            self.channel_attention_map = x

        return self.multiply([input, x])

In [4]:
class Spatial_Attention(layers.Layer):
    def __init__(self, kernel_size, cbam_id):
        super().__init__(name=f"cbam{cbam_id}_spatial")

        # Property for saving last attention map
        self.spatial_attention_map = None

        self.conv = Conv2D(1, kernel_size, strides=1, padding='same', activation='sigmoid', name=f"cbam{cbam_id}_spatial_conv")
        
        self.concat = Concatenate(axis=-1, name=f"cbam{cbam_id}_spatial_concatenate")
        self.multiply = Multiply(name=f"cbam{cbam_id}_spatial_multiply")

        self.attention = None # Value of spatial attention weights after last call()

    def build(self, input_shape):
        x_shape = (input_shape[0], input_shape[1], input_shape[2], 2)
        self.conv.build(x_shape)


    def compute_output_shape(self, input_shape):
        return input_shape

    def call(self, input, save_attention=False):
        x = input

        avg_pool = tf.reduce_mean(x, axis=-1, keepdims=True)
        max_pool = tf.reduce_max(x, axis=-1, keepdims=True)

        x = self.concat([avg_pool, max_pool])

        x = self.conv(x)

        if save_attention:
            self.spatial_attention_map = x
        
        return self.multiply([input, x])

In [5]:
class CBAM(layers.Layer):
    def __init__(self, channels, cbam_id, reduction=8, kernel_size=7):
        super().__init__(name=f"CBAM_{cbam_id}")

        self.channel = Channel_Attention(channels, reduction, cbam_id=cbam_id)
        self.spatial = Spatial_Attention(kernel_size, cbam_id=cbam_id)

    def build(self, input_shape):
        self.channel.build(input_shape)
        self.spatial.build(input_shape)

    def compute_output_shape(self, input_shape):
        return input_shape

    def call(self, input, save_attention=False):
        x = input

        x = self.channel(x, save_attention=save_attention)
        x = self.spatial(x, save_attention=save_attention)

        return x

## Model definition

In [6]:
@keras.saving.register_keras_serializable(package="Custom")
class CBAM_Model(keras.Model):
    def __init__(self, name="CbamModel", **kwargs):
        super().__init__(**kwargs)
        self.name = name
        
        # VGG19
        # Block 1
        self.vgg19_block1_conv1 = Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv1")
        self.vgg19_block1_conv2 = Conv2D(64, (3, 3), activation="relu", padding="same", name="block1_conv2")
        self.vgg19_block1_attention = CBAM(64, cbam_id='block1')
        self.vgg19_block1_pool = MaxPooling2D((2, 2), strides=(2, 2), name="block1_pool")

        # Block 2
        self.vgg19_block2_conv1 = Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv1")
        self.vgg19_block2_conv2 = Conv2D(128, (3, 3), activation="relu", padding="same", name="block2_conv2")
        self.vgg19_block2_attention = CBAM(128, cbam_id='block2')
        self.vgg19_block2_pool = MaxPooling2D((2, 2), strides=(2, 2), name="block2_pool")

        # Block 3
        self.vgg19_block3_conv1 = Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv1")
        self.vgg19_block3_conv2 = Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv2")
        self.vgg19_block3_conv3 = Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv3")
        self.vgg19_block3_conv4 = Conv2D(256, (3, 3), activation="relu", padding="same", name="block3_conv4")
        self.vgg19_block3_attention = CBAM(256, cbam_id='block3')
        self.vgg19_block3_pool = MaxPooling2D((2, 2), strides=(2, 2), name="block3_pool")
        
        # Block 4
        self.vgg19_block4_conv1 = Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv1")
        self.vgg19_block4_conv2 = Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv2")
        self.vgg19_block4_conv3 = Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv3")
        self.vgg19_block4_conv4 = Conv2D(512, (3, 3), activation="relu", padding="same", name="block4_conv4")
        self.vgg19_block4_attention = CBAM(512, cbam_id='block4')
        self.vgg19_block4_pool = MaxPooling2D((2, 2), strides=(2, 2), name="block4_pool")

        # Block 5
        self.vgg19_block5_conv1 = Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv1")
        self.vgg19_block5_conv2 = Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv2")
        self.vgg19_block5_conv3 = Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv3")
        self.vgg19_block5_conv4 = Conv2D(512, (3, 3), activation="relu", padding="same", name="block5_conv4")
        self.vgg19_block5_attention = CBAM(512, cbam_id='block5')
        self.vgg19_block5_pool = MaxPooling2D((2, 2), strides=(2, 2), name="block5_pool")

        # Classifier layer
        self.class_flatten = Flatten(name="class_flatten")
        
        self.class_dense1 = Dense(512, activation = 'relu', name="class_dense1")
        self.class_dropout1 = Dropout(0.4, name="class_dropout1")
        self.class_batch1 = BatchNormalization(name="class_batch1")
        
        self.class_dense2 = Dense(512, activation = 'relu', name="class_dense2")
        self.class_dropout2 = Dropout(0.3, name="class_dropout2")
        self.class_batch2 = BatchNormalization(name="class_batch2")

        self.class_dense3 = Dense(LABEL_CLASS, activation = 'softmax', name="class_dense3")

    def call(self, inputs, save_attention=False, training=False):
        # VGG19
        # Block 1
        x = self.vgg19_block1_conv1(inputs)
        x = self.vgg19_block1_conv2(x)
        x = self.vgg19_block1_attention(x, save_attention=save_attention)
        x = self.vgg19_block1_pool(x)

        # Block 2
        x = self.vgg19_block2_conv1(x)
        x = self.vgg19_block2_conv2(x)
        x = self.vgg19_block2_attention(x, save_attention=save_attention)
        x = self.vgg19_block2_pool(x)

        # Block 3
        x = self.vgg19_block3_conv1(x)
        x = self.vgg19_block3_conv2(x)
        x = self.vgg19_block3_conv3(x)
        x = self.vgg19_block3_conv4(x)
        x = self.vgg19_block3_attention(x, save_attention=save_attention)
        x = self.vgg19_block3_pool(x)

        # Block 4
        x = self.vgg19_block4_conv1(x)
        x = self.vgg19_block4_conv2(x)
        x = self.vgg19_block4_conv3(x)
        x = self.vgg19_block4_conv4(x)
        x = self.vgg19_block4_attention(x, save_attention=save_attention)
        x = self.vgg19_block4_pool(x)

        # Block 5
        x = self.vgg19_block5_conv1(x)
        x = self.vgg19_block5_conv2(x)
        x = self.vgg19_block5_conv3(x)
        x = self.vgg19_block5_conv4(x)
        x = self.vgg19_block5_attention(x, save_attention=save_attention)
        x = self.vgg19_block5_pool(x)

        # Classifier layer
        x = self.class_flatten(x)

        x = self.class_dense1(x)
        x = self.class_dropout1(x, training=training)
        x = self.class_batch1(x, training=training)

        x = self.class_dense2(x)
        x = self.class_dropout2(x, training=training)
        x = self.class_batch2(x, training=training)

        x = self.class_dense3(x)

        return x

    def build(self, input_shape):
        # VGG19
        # Block 1
        self.vgg19_block1_conv1.build(input_shape)
        x_shape = self.vgg19_block1_conv1.compute_output_shape(input_shape)
        self.vgg19_block1_conv2.build(x_shape)
        x_shape = self.vgg19_block1_conv2.compute_output_shape(x_shape)
        self.vgg19_block1_attention.build(x_shape)
        x_shape = self.vgg19_block1_attention.compute_output_shape(x_shape)
        self.vgg19_block1_pool.build(x_shape)
        x_shape = self.vgg19_block1_pool.compute_output_shape(x_shape)

        # Block 2
        self.vgg19_block2_conv1.build(x_shape)
        x_shape = self.vgg19_block2_conv1.compute_output_shape(x_shape)
        self.vgg19_block2_conv2.build(x_shape)
        x_shape = self.vgg19_block2_conv2.compute_output_shape(x_shape)
        self.vgg19_block2_attention.build(x_shape)
        x_shape = self.vgg19_block2_attention.compute_output_shape(x_shape)
        self.vgg19_block2_pool.build(x_shape)
        x_shape = self.vgg19_block2_pool.compute_output_shape(x_shape)

        # Block 3
        self.vgg19_block3_conv1.build(x_shape)
        x_shape = self.vgg19_block3_conv1.compute_output_shape(x_shape)
        self.vgg19_block3_conv2.build(x_shape)
        x_shape = self.vgg19_block3_conv2.compute_output_shape(x_shape)
        self.vgg19_block3_conv3.build(x_shape)
        x_shape = self.vgg19_block3_conv3.compute_output_shape(x_shape)
        self.vgg19_block3_conv4.build(x_shape)
        x_shape = self.vgg19_block3_conv4.compute_output_shape(x_shape)
        self.vgg19_block3_attention.build(x_shape)
        x_shape = self.vgg19_block3_attention.compute_output_shape(x_shape)
        self.vgg19_block3_pool.build(x_shape)
        x_shape = self.vgg19_block3_pool.compute_output_shape(x_shape)

        # Block 4
        self.vgg19_block4_conv1.build(x_shape)
        x_shape = self.vgg19_block4_conv1.compute_output_shape(x_shape)
        self.vgg19_block4_conv2.build(x_shape)
        x_shape = self.vgg19_block4_conv2.compute_output_shape(x_shape)
        self.vgg19_block4_conv3.build(x_shape)
        x_shape = self.vgg19_block4_conv3.compute_output_shape(x_shape)
        self.vgg19_block4_conv4.build(x_shape)
        x_shape = self.vgg19_block4_conv4.compute_output_shape(x_shape)
        self.vgg19_block4_attention.build(x_shape)
        x_shape = self.vgg19_block4_attention.compute_output_shape(x_shape)
        self.vgg19_block4_pool.build(x_shape)
        x_shape = self.vgg19_block4_pool.compute_output_shape(x_shape)

        # Block 5
        self.vgg19_block5_conv1.build(x_shape)
        x_shape = self.vgg19_block5_conv1.compute_output_shape(x_shape)
        self.vgg19_block5_conv2.build(x_shape)
        x_shape = self.vgg19_block5_conv2.compute_output_shape(x_shape)
        self.vgg19_block5_conv3.build(x_shape)
        x_shape = self.vgg19_block5_conv3.compute_output_shape(x_shape)
        self.vgg19_block5_conv4.build(x_shape)
        x_shape = self.vgg19_block5_conv4.compute_output_shape(x_shape)
        self.vgg19_block5_attention.build(x_shape)
        x_shape = self.vgg19_block5_attention.compute_output_shape(x_shape)
        self.vgg19_block5_pool.build(x_shape)
        x_shape = self.vgg19_block5_pool.compute_output_shape(x_shape)

        # Classifier layer
        self.class_flatten.build(x_shape)
        x_shape = self.class_flatten.compute_output_shape(x_shape)
        self.class_dense1.build(x_shape)
        x_shape = self.class_dense1.compute_output_shape(x_shape)
        self.class_dropout1.build(x_shape)
        x_shape = self.class_dropout1.compute_output_shape(x_shape)
        self.class_batch1.build(x_shape)
        x_shape = self.class_batch1.compute_output_shape(x_shape)
        
        self.class_dense2.build(x_shape)
        x_shape = self.class_dense2.compute_output_shape(x_shape)
        self.class_dropout2.build(x_shape)
        x_shape = self.class_dropout2.compute_output_shape(x_shape)
        self.class_batch2.build(x_shape)
        x_shape = self.class_batch2.compute_output_shape(x_shape)

        self.class_dense3.build(x_shape)

## Building the model

In [7]:
model = CBAM_Model()
model.build((None, IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))
model.summary()

Model: "CbamModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv2D)                │ (None, 64, 64, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 64, 64, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ CBAM_block1 (CBAM)                   │ (None, 64, 64, 64)          │           1,195 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 32, 32, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ CBAM_block2 (CBAM)                   │ (None, 32, 32, 128)         │           4,339 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 16, 16, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 16, 16, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 16, 16, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, 16, 16, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ CBAM_block3 (CBAM)                   │ (None, 16, 16, 256)         │          16,771 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 8, 8, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ CBAM_block4 (CBAM)                   │ (None, 8, 8, 512)           │          66,211 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 21,508,289 (82.05 MB)

 Trainable params: 21,506,241 (82.04 MB)

 Non-trainable params: 2,048 (8.00 KB)

## Loading the weights

In [8]:
model.load_weights(WEIGHTS, by_name=True, skip_mismatch=True)

## Compiling the model

In [9]:
model.compile(optimizer=optimizers.RMSprop(learning_rate=LEARNINGRATE), loss='categorical_crossentropy', metrics=['accuracy'])

## Saving the clean model

In [11]:
save_model(model, "CleanModels\\CleanCbamModel.keras", include_optimizer=True)

## Loading the dataset

In [12]:
def get_labels_images(path):
    labels = []
    images = []
    directories = []
    
    for directory in os.listdir(path):
        for Label in os.listdir(path + '/' + directory):
            for Image in os.listdir(path + '/' + directory + '/' + Label):
                directories.append(directory)
                labels.append(Label)
                images.append(directory + '/' + Label + '/' + Image)
                
    return pd.DataFrame({'directories':directories, 'labels':labels, 'images':images})

In [13]:
df = get_labels_images(dataset_dir)

## Preparing dataset for training

In [14]:
def split_data(data):
    train_df, test_df = train_test_split(data, test_size=0.10, random_state=SEED)
    train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=SEED)

    train_df = train_df.reset_index(drop=True)
    val_df = val_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)

    print('----------------------------------------------------------------')
    print("The Number of Samples per Split")
    print('----------------------------------------------------------------')
    print('Number of   training samples : {}'.format(train_df.shape[0]))
    print('Number of validation samples : {}'.format(val_df.shape[0]))
    print('Number of       test samples : {}'.format(test_df.shape[0]))
    print('----------------------------------------------------------------')

    return train_df, val_df, test_df

In [15]:
def define_image_generators(train_df, val_df, test_df):
    train_generator = datagen.flow_from_dataframe(train_df, **data_gen_args)
    val_generator = datagen.flow_from_dataframe(val_df, **data_gen_args)
    test_generator = datagen.flow_from_dataframe(test_df, **data_gen_args, shuffle = False)
    
    return train_generator, val_generator, test_generator

In [16]:
train_df, val_df, test_df = split_data(df)

----------------------------------------------------------------
The Number of Samples per Split
----------------------------------------------------------------
Number of   training samples : 333870
Number of validation samples : 58919
Number of       test samples : 43644
----------------------------------------------------------------


In [17]:
train_generator, val_generator, test_generator = define_image_generators(train_df, val_df, test_df)

Found 333870 validated image filenames belonging to 26 classes.
Found 58919 validated image filenames belonging to 26 classes.
Found 43644 validated image filenames belonging to 26 classes.


## Loading model from the checkpoints

In [ ]:
# Loading the latest checkpoint
latest_checkpoint = max(glob.glob(os.path.join(checkpoints_dir, "cp-*")), key=os.path.getmtime)
start_epoch = int(re.search(r"cp-(\d+)", latest_checkpoint).group(1))

# Loading full model from checkpoints
model = tf.keras.models.load_model(latest_checkpoint)

## Training model

In [ ]:
checkpoints = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoints_path,
    verbose=1,
    save_weights_only=False,
    save_freq="epoch"
)

callback_list = [early_stop, checkpoints, learningrate_reduction]

In [18]:
history = model.fit(train_generator,
                    epochs = EPOCHS,
                    initial_epoch=start_epoch
                    validation_data = val_generator,
                    callbacks = callback_list)

C:\Users\peklu\Desktop\VisualAttentionAsExplanation\notebooks\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1334/2609 ━━━━━━━━━━━━━━━━━━━━ 5:11:17 15s/step - accuracy: 0.4306 - loss: 2.1935

KeyboardInterrupt: 

## Saving trained model and statistics

In [ ]:
save_model(model, "TrainedModels\\TrainedCbamModel.keras", include_optimizer=True)

In [ ]:
with open('..\\Statistics\\TrainingStatistics\\CbamTraining.pkl', 'wb') as file:
    pickle.dump(history.history, file)